In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-w8orbtkv
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-w8orbtkv
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10733 sha256=25d7336482e00933626da974c4579936e1bcd2169b147b54a7d099795d56a4e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4n4vx9z/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpqo0gwia_".


2D lid-driven cavity problem

In [5]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <fstream>

// Grid and model parameters
const int nx = 16;         // Number of grid points in x
const int ny = 16;         // Number of grid points in y
const int q = 9;            // D2Q9 model
const int maxIter = 10000;  // Maximum number of iterations
const double uLid = 0.1;    // Lid velocity
const double tau = 0.6;     // Relaxation time
const double rho0 = 1.0;    // Initial density

// D2Q9 constants
__constant__ int ex[q] = {0, 1, 0, -1, 0, 1, -1, -1, 1};
__constant__ int ey[q] = {0, 0, 1, 0, -1, 1, 1, -1, -1};
__constant__ double w[q] = {4.0 / 9.0, 1.0 / 9.0, 1.0 / 9.0, 1.0 / 9.0, 1.0 / 9.0,
                            1.0 / 36.0, 1.0 / 36.0, 1.0 / 36.0, 1.0 / 36.0};

// Kernel for collision step
__global__ void collisionStep(double* f, double* rho, double* ux, double* uy, double* fEq, double omega, int nx, int ny) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    if (i >= nx || j >= ny) return;

    int idx = j * nx + i;
    double uSqr = ux[idx] * ux[idx] + uy[idx] * uy[idx];

    for (int k = 0; k < q; k++) {
        double eU = ex[k] * ux[idx] + ey[k] * uy[idx];
        fEq[idx * q + k] = w[k] * rho[idx] * (1.0 + 3.0 * eU + 4.5 * eU * eU - 1.5 * uSqr);
        f[idx * q + k] = (1 - omega) * f[idx * q + k] + omega * fEq[idx * q + k];
    }
}

// Kernel for streaming step with bounce-back boundary
__global__ void streamingStep(double* f, double* fTemp, int nx, int ny) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    if (i >= nx || j >= ny) return;

    for (int k = 0; k < q; k++) {
        int x = i - ex[k];
        int y = j - ey[k];

        // Apply bounce-back boundary conditions
        if (x < 0 || x >= nx || y < 0 || y >= ny) {
            fTemp[(j * nx + i) * q + k] = f[(j * nx + i) * q + (k ^ 3)];
        } else {
            fTemp[(j * nx + i) * q + k] = f[(y * nx + x) * q + k];
        }
    }
}

// Kernel for computing macroscopic variables
__global__ void computeMacros(double* f, double* rho, double* ux, double* uy, int nx, int ny) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    if (i >= nx || j >= ny) return;

    int idx = j * nx + i;
    rho[idx] = 0.0;
    ux[idx] = 0.0;
    uy[idx] = 0.0;

    for (int k = 0; k < q; k++) {
        rho[idx] += f[idx * q + k];
        ux[idx] += f[idx * q + k] * ex[k];
        uy[idx] += f[idx * q + k] * ey[k];
    }
    ux[idx] /= rho[idx];
    uy[idx] /= rho[idx];
}

// Kernel for applying boundary conditions
__global__ void applyBoundaryConditions(double* f, double* ux, double* uy, double uLid, int nx, int ny) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;

    // Top lid boundary (moving lid)
    if (i < nx) {
        int idx = (ny - 1) * nx + i;
        ux[idx] = uLid;
        uy[idx] = 0.0;
        for (int k = 0; k < q; k++) {
            if (ey[k] == -1) { // Downward moving direction
                f[idx * q + k] = f[idx * q + (k ^ 3)];
            }
        }
    }

    // Bottom wall (no-slip)
    if (i < nx) {
        int idx = i; // Bottom row
        ux[idx] = 0.0;
        uy[idx] = 0.0;
        for (int k = 0; k < q; k++) {
            f[idx * q + k] = f[idx * q + (k ^ 3)];
        }
    }

    // Left and right walls (no-slip)
    if (i < ny) {
        int idxLeft = i * nx;       // Left wall
        int idxRight = i * nx + nx - 1; // Right wall

        ux[idxLeft] = 0.0;
        uy[idxLeft] = 0.0;
        ux[idxRight] = 0.0;
        uy[idxRight] = 0.0;

        for (int k = 0; k < q; k++) {
            f[idxLeft * q + k] = f[idxLeft * q + (k ^ 3)];
            f[idxRight * q + k] = f[idxRight * q + (k ^ 3)];
        }
    }
}

// Host function
void runLBM() {
    size_t size = nx * ny * q * sizeof(double);
    size_t macroSize = nx * ny * sizeof(double);

    std::vector<double> f(nx * ny * q, 0.0);
    std::vector<double> rho(nx * ny, rho0);
    std::vector<double> ux(nx * ny, 0.0);
    std::vector<double> uy(nx * ny, 0.0);

    double *d_f, *d_rho, *d_ux, *d_uy, *d_fEq, *d_fTemp;
    cudaMalloc(&d_f, size);
    cudaMalloc(&d_rho, macroSize);
    cudaMalloc(&d_ux, macroSize);
    cudaMalloc(&d_uy, macroSize);
    cudaMalloc(&d_fEq, size);
    cudaMalloc(&d_fTemp, size);

    cudaMemcpy(d_f, f.data(), size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_rho, rho.data(), macroSize, cudaMemcpyHostToDevice);

    dim3 blockSize(16, 16);
    dim3 gridSize((nx + blockSize.x - 1) / blockSize.x, (ny + blockSize.y - 1) / blockSize.y);

    double omega = 1.0 / tau;

    for (int iter = 0; iter < maxIter; iter++) {
        collisionStep<<<gridSize, blockSize>>>(d_f, d_rho, d_ux, d_uy, d_fEq, omega, nx, ny);
        streamingStep<<<gridSize, blockSize>>>(d_f, d_fTemp, nx, ny);
        computeMacros<<<gridSize, blockSize>>>(d_fTemp, d_rho, d_ux, d_uy, nx, ny);
        applyBoundaryConditions<<<gridSize, blockSize>>>(d_fTemp, d_ux, d_uy, uLid, nx, ny);
        std::swap(d_f, d_fTemp);
    }

    cudaMemcpy(f.data(), d_f, size, cudaMemcpyDeviceToHost);

    cudaFree(d_f);
    cudaFree(d_rho);
    cudaFree(d_ux);
    cudaFree(d_uy);
    cudaFree(d_fEq);
    cudaFree(d_fTemp);

    std::cout << "Simulation completed.\n";
}

int main() {
    runLBM();
    return 0;
}

Simulation completed.



3 dimension problem

In [7]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <fstream>
/**
 * @author     li yibo
 * @date      2024-12-10
 * @brief     This program implements of 2D and 3D Lattice Boltzmann Method (LBM).
 * @to do     Add an obstacle in the domain and compute the lift and drag
 * @detail     main idea is present the whole cavity as a 2/3 dimension matrix and basic unit of each entry in matrix
 *         have his velocity in 2/3D and weight to connected to other unit.So we can establish collisionStep, streamingStep and
 *         boundaryCondition these three func to simulate the process of fluid evolution. And compute the velocity by
 *         distribution function ->f<-(what we compute in three functions mentioned above) and density.
 */

// Parameters
const int nx = 16;         // Grid size in x
const int ny = 16;         // Grid size in y
const int nz = 16;         // Grid size in z
// 这里我测试了128*128*128 然后发现需要很久 2维5秒这里可能需要32分钟 所以可以改改维度 比如16*16*16
const int q = 27;            // D3Q27 model
const int maxIter = 10000;  // Number of iterations
const double uLid = 0.1;    // Lid velocity
const double Re = 100.0;    // Reynolds number
const double nu = uLid * nx / Re;  // Viscous number
const double tau = 3.0 * nu + 0.5;  // Relaxation time
const double rho0 = 1.0;    // Initial density

// D3Q27 Lattice parameters
__constant__ int ex[27] = {1, 0, -1, -1, -1, 0, 1, 1, 0,   1, 0, -1, -1, 0, 1, 1, 0, -1,   0, -1, -1, 0, 1, 1, -1, 0, -1};
__constant__ int ey[27] = {1, 1, 1, 0, -1, -1, -1, 0, 0,   1, 1, 1, 0, 0, 0, -1, -1, -1,   0, 0, 1, 1, 1, 0, -1, -1, -1};
__constant__ int ez[27] = {1, 1, 1, 1, 1, 1, 1, 1, 1,   0, 0, 0, 0, 0, 0, 0, 0, 0,   -1, -1, -1, -1, -1, -1, -1, -1, -1};

__constant__ double w[q] = {1.0/216.0, 1.0/54.0, 1.0/216.0, 1.0/54.0, 1.0/216.0, 1.0/54.0, 1.0/216.0, 1.0/54.0, 2.0/27.0,
                1.0/54.0, 2.0/27.0, 1.0/54.0, 2.0/27.0, 8.0/27.0, 2.0/27.0, 1.0/54.0, 2.0/27.0, 1.0/54.0,
                2.0/27.0, 1.0/54.0, 1.0/216.0, 1.0/54.0, 1.0/216.0, 1.0/54.0, 1.0/216.0, 1.0/54.0, 1.0/216.0,};

// Collision step
__global__ void collisionStep(double* f, double* rho, double* ux, double* uy, double* uz, double* fEq, double omega, int nx, int ny, int nz) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    int k = threadIdx.z + blockIdx.z * blockDim.z;
    if (i >= nx || j >= ny ||k >= nz) return;

    int idx = k * nx * ny + j * nx + i;
    double uSqr = ux[idx] * ux[idx] + uy[idx] * uy[idx] + uz[idx] * uz[idx];

    for (int t = 0; t < q; t++) {
        double eU = ex[t] * ux[idx] + ey[t] * uy[idx] + ez[t] * uz[idx];
        fEq[idx * q + t] = w[t] * rho[idx] * (1.0 + 3.0 * eU + 4.5 * eU * eU - 1.5 * uSqr);
        f[idx * q + t] = (1 - omega) * f[idx * q + t] + omega * fEq[idx * q + t];
        // 向平衡方向偏移 偏移尺度是ω
    }
}

// Streaming step
__global__ void streamingStep(double* f, double* fTemp, int nx, int ny, int nz) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    int k = threadIdx.z + blockIdx.z * blockDim.z;
    if (i >= nx || j >= ny ||k >= nz) return;

    for (int t = 0; t < q; t++) {
        int x = i - ex[t];
        int y = j - ey[t];
        int z = k - ez[t];
        int srcIdx = z * nx * ny + y * nx + x;
        if (x < 0 || x >= nx || y < 0 || y >= ny || z < 0 || z >= nz) {
            fTemp[(k * nx * ny +j * nx + i) * q + t] = f[(k * nx * ny +j * nx + i) * q + (26 - t)];  // 边界取反方向
        } else {
            fTemp[(k * nx * ny +j * nx + i) * q + t] = f[(z * nx * ny + y * nx + x) * q + t];  // 从相邻格子读对应方向
        }
    }
}

// Compute macroscopic variables
__global__ void computeMacros(double* f, double* rho, double* ux, double* uy, double* uz, int nx, int ny, int nz) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    int k = threadIdx.z + blockIdx.z * blockDim.z;
    if (i >= nx || j >= ny ||k >= nz) return;

    int idx = k * nx * ny + j * nx + i;
    rho[idx] = 0.0;
    ux[idx] = 0.0;
    uy[idx] = 0.0;
    uz[idx] = 0.0;

    for (int t = 0; t < q; t++) {
        rho[idx] += f[idx * q + t];
        ux[idx] += f[idx * q + t] * ex[t];
        uy[idx] += f[idx * q + t] * ey[t];
        uz[idx] += f[idx * q + t] * ez[t];
    }
    ux[idx] /= rho[idx];
    uy[idx] /= rho[idx];
}

// Apply boundary conditions
__global__ void applyBoundaryConditions(double* f, double* ux, double* uy, double* uz, double uLid, int nx, int ny, int nz) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    int k = threadIdx.z + blockIdx.z * blockDim.z;
    if (i >= nx || j >= ny || k >= nz) return;

    int idx = (k * ny * nx) + (j * nx) + i;
    // 1. 顶部边界 (z = nz - 1, Lid-Driven Boundary)
    if (k == nz - 1) {
        ux[idx] = uLid;
        uy[idx] = 0.0;
        uz[idx] = 0.0;
        for (int q = 0; q < 27; q++) {
            if (ez[q] == -1) { // 向下移动
                f[idx * 27 + q] = f[idx * 27 + (26 - q)];
            }
        }
    }
    // 底
    if (k == 0) {
        ux[idx] = 0.0;
        uy[idx] = 0.0;
        uz[idx] = 0.0;
        for (int q = 0; q < 27; q++) {
            if (ez[q] == 1) { // 向上
                f[idx * 27 + q] = f[idx * 27 + (26 - q)];
            }
        }
    }
    // 前
    if (j == ny - 1) {
        ux[idx] = 0.0;
        uy[idx] = 0.0;
        uz[idx] = 0.0;
        for (int q = 0; q < 27; q++) {
            if (ey[q] == -1) {
                f[idx * 27 + q] = f[idx * 27 + (26 - q)];
            }
        }
    }
    // 后
    if (j == 0) {
        ux[idx] = 0.0;
        uy[idx] = 0.0;
        uz[idx] = 0.0;
        for (int q = 0; q < 27; q++) {
            if (ey[q] == 1) {
                f[idx * 27 + q] = f[idx * 27 + (26 - q)];
            }
        }
    }
    // 左
    if (i == 0) {
        ux[idx] = 0.0;
        uy[idx] = 0.0;
        uz[idx] = 0.0;
        for (int q = 0; q < 27; q++) {
            if (ex[q] == 1) {
                f[idx * 27 + q] = f[idx * 27 + (26 - q)];
            }
        }
    }
    // 右
    if (i == nx - 1) {
        ux[idx] = 0.0;
        uy[idx] = 0.0;
        uz[idx] = 0.0;
        for (int q = 0; q < 27; q++) {
            if (ex[q] == -1) {
                f[idx * 27 + q] = f[idx * 27 + (26 - q)];
            }
        }
    }
}


// Host function
void runLBM() {
    size_t size = nx * ny * nz *q * sizeof(double);
    size_t macroSize = nx * ny * nz * sizeof(double);

    // Host variables
    std::vector<double> f(nx * nz * ny * q, 0.0);
    std::vector<double> rho(nx * nz * ny, rho0);
    std::vector<double> ux(nx * nz * ny, 0.0);
    std::vector<double> uy(nx * nz * ny, 0.0);

    // Device variables
    double *d_f, *d_rho, *d_ux, *d_uy, *d_uz,*d_fEq, *d_fTemp;
    cudaMalloc(&d_f, size);
    cudaMalloc(&d_rho, macroSize);
    cudaMalloc(&d_ux, macroSize);
    cudaMalloc(&d_uy, macroSize);
    cudaMalloc(&d_uz, macroSize);
    cudaMalloc(&d_fEq, size);
    cudaMalloc(&d_fTemp, size);

    cudaMemcpy(d_f, f.data(), size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_rho, rho.data(), macroSize, cudaMemcpyHostToDevice);

    dim3 blockSize(16, 16, 4);
    dim3 gridSize((nx + blockSize.x - 1) / blockSize.x, (ny + blockSize.y - 1) / blockSize.y, (nz + blockSize.z -1)/ blockSize.z);

    double omega = 1.0 / tau;

    for (int iter = 0; iter < maxIter; iter++) {
        collisionStep<<<gridSize, blockSize>>>(d_f, d_rho, d_ux, d_uy, d_uz, d_fEq, omega, nx, ny, nz);
        streamingStep<<<gridSize, blockSize>>>(d_f, d_fTemp, nx, ny, nz);
        computeMacros<<<gridSize, blockSize>>>(d_fTemp, d_rho, d_ux, d_uy, d_uz, nx, ny, nz);
        applyBoundaryConditions<<<gridSize, blockSize>>>(d_f, d_ux, d_uy, d_uz, uLid, nx, ny, nz);
        std::swap(d_f, d_fTemp);
    }

    cudaMemcpy(f.data(), d_f, size, cudaMemcpyDeviceToHost);

    cudaFree(d_f);
    cudaFree(d_rho);
    cudaFree(d_ux);
    cudaFree(d_uy);
    cudaFree(d_uz);
    cudaFree(d_fEq);
    cudaFree(d_fTemp);

    std::cout << "Simulation completed.\n";
}

int main() {
    runLBM();
    return 0;
}

Simulation completed.

